In [2]:
import cv2
import numpy as np
import os


In [4]:
# Load the pre-trained face detection model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
# Set the desired image size for training
target_size = (120, 120)  # Adjust this as per your requirements

In [5]:
# Load the dataset and train the face recognizer
dataset_dir = 'dataset'
data = []
labels = []
model = cv2.face.EigenFaceRecognizer_create()

In [6]:

for person_dir in os.listdir(dataset_dir):
    person_path = os.path.join(dataset_dir, person_dir)
    if os.path.isdir(person_path):
        label = person_dir
        label_id = labels.index(label) if label in labels else len(labels)
        if label not in labels:
            labels.append(label)
        for img_path in os.listdir(person_path):
            img_path = os.path.join(person_path, img_path)
            img = cv2.imread(img_path, 0)  # Load grayscale image
            img = cv2.resize(img, target_size)  # Resize the image to the target size
            data.append(img)
            labels_array = np.full(len(data), label_id, dtype=np.int32)


In [7]:

model.train(data, np.array(labels_array))


In [8]:

# Open the camera
cap = cv2.VideoCapture(0)
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # For each detected face
    for (x, y, w, h) in faces:
        # Extract the face region
        face = gray[y:y+h, x:x+w]
        face = cv2.resize(face, target_size)  # Resize the face region to the target size

        # Compare the face with the trained model
        label, confidence = model.predict(face)
        similarity = 1 - (confidence / 100)  # Convert confidence to similarity

        # If the similarity is high enough (e.g., above 0.7), consider it a match
        if similarity > 0.3:
            print(f"Face matched: {labels[label]}")
            color = (0, 255, 0)  # Green color for a match
            text = labels[label]  # Display the recognized name
        else:
            color = (0, 0, 255)  # Red color for no match
            text = "Unknown"  # Display "Unknown" for a non-match

        # Draw a rectangle around the face
        stroke = 2
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, stroke)

        # Draw the text below the rectangle
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.5
        text_size, _ = cv2.getTextSize(text, font, font_scale, 1)
        text_x = x
        text_y = y + h + text_size[1] + 5
        cv2.putText(frame, text, (text_x, text_y), font, font_scale, color, 1, cv2.LINE_AA)

    # Display the resulting frame
    cv2.imshow('Face Recognition', frame)

    # Exit the loop if 'q' is pressed
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break

# Release the capture and close the window
cap.release()
cv2.destroyAllWindows()